<a href="https://colab.research.google.com/github/gkunal8019/Cointab-Assignment/blob/main/Cointab_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import numpy as np

In [29]:

# Load the LHS data (X's internal data)
order_report = pd.read_excel("https://archive.org/download/company-x-order-report/Company%20X%20-%20Order%20Report.xlsx")
sku_master = pd.read_excel("https://archive.org/download/company-x-order-report/Company%20X%20-%20SKU%20Master.xlsx")
warehouse_mapping = pd.read_excel("https://archive.org/download/company-x-order-report/Company%20X%20-%20Pincode%20Zones.xlsx")

# Load the RHS data (courier company invoice)
courier_invoice = pd.read_excel("https://archive.org/download/company-x-order-report/Courier%20Company%20-%20Invoice.xlsx")
courier_rate_card = pd.read_excel("https://archive.org/download/company-x-order-report/Courier%20Company%20-%20Rates.xlsx")

In [21]:
print("Column names in order_report:", order_report.columns)
print("Column names in sku_master:", sku_master.columns)
print("Column names in warehouse_mapping:", warehouse_mapping.columns)


Column names in order_report: Index(['ExternOrderNo', 'SKU', 'Order Qty'], dtype='object')
Column names in sku_master: Index(['SKU', 'Weight (g)'], dtype='object')
Column names in warehouse_mapping: Index(['warehouse_mapping', 'Customer Pincode', 'Zone'], dtype='object')


In [37]:
warehouse_mapping.head(3)

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d


In [77]:
# Merge the LHS data to create a single data frame with all the information
merged_data = pd.merge(order_report, sku_master, on="SKU", how="left")
merged_data = pd.merge(merged_data, warehouse_mapping, left_on="ExternOrderNo",right_on="Customer Pincode", how="left")

In [100]:
merged_data['ExternOrderNo'].value_counts()

2001809820    14
2001827036     9
2001812854     8
2001821185     8
2001817093     8
              ..
2001814580     1
2001820690     1
2001806686     1
2001806823     1
2001807012     1
Name: ExternOrderNo, Length: 124, dtype: int64

In [83]:
merged_data

,ExternOrderNo,SKU,Order Qty,Weight (g),Warehouse Pincode,Customer Pincode,Zone,Total Weight,Total Weight (KG),Weight Slab (KG)
0,2001827036,8904223818706,1.0,127,NaN,NaN,NaN,127.0,0.5,1.0
1,2001827036,8904223819093,1.0,150,NaN,NaN,NaN,150.0,0.5,1.0
2,2001827036,8904223819109,1.0,100,NaN,NaN,NaN,100.0,0.5,1.0
3,2001827036,8904223818430,1.0,165,NaN,NaN,NaN,165.0,0.5,1.0
4,2001827036,8904223819277,1.0,350,NaN,NaN,NaN,350.0,0.5,1.0
...,...,...,...,...,...,...,...,...,...,...
396,2001806229,8904223818942,1.0,133,NaN,NaN,NaN,133.0,0.5,1.0
397,2001806229,8904223818850,1.0,240,NaN,NaN,NaN,240.0,0.5,1.0
398,2001806226,8904223818850,2.0,240,NaN,NaN,NaN,480.0,0.5,1.0
399,2001806210,8904223816214,1.0,120,NaN,NaN,NaN,120.0,0.5,1.0


In [79]:
# Calculate the total weight of each order
merged_data["Total Weight"] = merged_data["Order Qty"] * merged_data["Weight (g)"]
merged_data["Total Weight (KG)"] = np.ceil(merged_data["Total Weight"] / 1000) / 2
# Calculate the weight slab
merged_data["Weight Slab (KG)"] = np.ceil(merged_data["Total Weight (KG)"])

In [80]:
# Group the merged data by Order ID to get the total weight and delivery zone for each order
grouped_data = merged_data.groupby("ExternOrderNo", as_index=False).agg({"Total Weight (KG)": "sum", "Warehouse Pincode": "first"})

In [104]:
courier_invoice

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0


In [106]:
# rename column 'old_column_name' to 'new_column_name'
grouped_data = grouped_data.rename(columns={'ExternOrderNo': 'Order ID'})

In [114]:
grouped_data

,Order ID,Total Weight (KG),Warehouse Pincode_x,AWB Code_x,Charged Weight_x,Warehouse Pincode_y,Customer Pincode_x,Zone_x,Type of Shipment_x,Billing Amount (Rs.)_x,...,Zone_y,Type of Shipment_y,Billing Amount (Rs.)_y,AWB Code,Charged Weight,Warehouse Pincode_y,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806232,1.0,NaN,1091117222124,1.30,121003,507101,d,Forward charges,135.0,...,d,Forward charges,135.0,1091117222124,1.30,121003,507101,d,Forward charges,135.0
1,2001806273,0.5,NaN,1091117222194,1.00,121003,486886,d,Forward charges,90.2,...,d,Forward charges,90.2,1091117222194,1.00,121003,486886,d,Forward charges,90.2
2,2001806408,1.5,NaN,1091117222931,2.50,121003,532484,d,Forward charges,224.6,...,d,Forward charges,224.6,1091117222931,2.50,121003,532484,d,Forward charges,224.6
3,2001806458,1.0,NaN,1091117223244,1.00,121003,143001,b,Forward charges,61.3,...,b,Forward charges,61.3,1091117223244,1.00,121003,143001,b,Forward charges,61.3
4,2001807012,1.0,NaN,1091117229345,0.15,121003,515591,d,Forward charges,45.4,...,d,Forward charges,45.4,1091117229345,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001812941,1.0,NaN,1091118551656,0.73,121003,325207,d,Forward charges,90.2,...,d,Forward charges,90.2,1091118551656,0.73,121003,325207,d,Forward charges,90.2
120,2001809383,1.0,NaN,1091117614452,0.50,121003,303702,d,Forward and RTO charges,86.7,...,d,Forward and RTO charges,86.7,1091117614452,0.50,121003,303702,d,Forward and RTO charges,86.7
121,2001820978,1.5,NaN,1091120922803,0.50,121003,313301,d,Forward charges,45.4,...,d,Forward charges,45.4,1091120922803,0.50,121003,313301,d,Forward charges,45.4
122,2001811475,2.5,NaN,1091121844806,0.50,121003,173212,b,Forward charges,33.0,...,b,Forward charges,33.0,1091121844806,0.50,121003,173212,b,Forward charges,33.0


In [111]:
# Merge the grouped data with the courier invoice data to get the expected charge and the delivery zone for each order
grouped_data = pd.merge(grouped_data, courier_invoice, on="Order ID", how="left")
#grouped_data = pd.merge(grouped_data, courier_rate_card, on=["Weight Slab (KG)", "Delivery Zone"], how="left")

# Calculate the expected charge
grouped_data["Expected Charge"] = grouped_data["Fixed"] + (grouped_data["Additional"] * (grouped_data["Total Weight (KG)"] - 0.5))

# Create the final data frame with the columns specified in the output
final_data = grouped_data[["Order ID", "AWB Number", "Total Weight (KG)", "Weight Slab (KG)", "Weight", "Weight Slab", "Delivery Zone", "Zone", "Expected Charge", "Charge", "Expected Charge - Charge"]]

# Save the final data frame to a CSV file
final_data.to_csv("final_data.csv", index=False)

# Create the summary table
correctly_charged = final_data[final_data["Expected Charge - Charge"] == 0]
summary = pd.DataFrame({"Count": [correctly_charged.shape[0]], "Amount (Rs.)": [correctly_charged["Charge"].sum()]})


<ipython-input-111-dc7128d4d3d5>:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Warehouse Pincode_x'} in the result is deprecated and will raise a MergeError in a future version.
  grouped_data = pd.merge(grouped_data, courier_invoice, on="Order ID", how="left")


KeyError: ignored